## Import modules

In [1]:
import pickle
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorchtools
import glob as gl
import random
import os
import time
import itertools

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.datasets import make_regression
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import log_loss
from numpy import hstack
from numpy import vstack

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings(action='ignore')

## Data loading

In [2]:
def find_directory(foldername, filename = None, back_num = 0):
    cur = os.getcwd()
    for i in range(back_num):
        cur = os.path.abspath(os.path.join(cur, os.pardir))
    for folder in foldername:
        cur = os.path.join(cur, folder)
    if not os.path.exists(cur):
        os.makedirs(cur)
        print(f'{cur} created')
    if filename != None:
        cur = os.path.join(cur, filename)
    return cur

os.getcwd()
find_directory(back_num = 1, foldername = ['Dataset'], filename = 'bat_dict.pkl')

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\All dataset'

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\Dataset\\bat_dict.pkl'

In [3]:
bat_dict_add_1_2 = find_directory(back_num = 2, foldername = ['Dataset'], filename = 'bat_1_2_dict_VITQcQd.pkl')
with open(bat_dict_add_1_2, 'rb') as tf:
    bat_dict_1_2 = pickle.load(tf)

bat_sel_dict = bat_dict_1_2
max_time = 120

In [4]:
# batch1: 0~40(41)
# batch2: 41~83(43)
# batch3: 64~123(40)

b1 = 0
b2 = 0
b3 = 0

for cell in bat_sel_dict.keys():
    if cell.startswith('b1'):
        b1 += 1
    elif cell.startswith('b2'):
        b2 += 1
    else:
        b3 += 1

print(b1, b2, b3)

# Tensor dimension: (cell, cycle, time, variable)
regression_cycles = [30, 50, 80]

# variable = [Qc, I, V, T, Qd]
num_variables = len(bat_sel_dict['b1c0']['1']['1.0'].keys())

for reg in regression_cycles:
    globals()[f'x_tensor_{reg}'] = torch.zeros(b1+b2+b3, reg, max_time, num_variables)
    cell_index = 1
    for cell in bat_sel_dict.keys():
        for cycle in range(1, reg+1):
            for time in bat_sel_dict[cell][str(cycle)].keys():
    #             print(bat_sel_dict[cell][str(cycle)][str(time)])
                value_list = [i for i in bat_sel_dict[cell][str(cycle)][str(time)].values()]
    #             print(value_list)
                globals()[f'x_tensor_{reg}'][cell_index-1, cycle-1, int(float(time)*max_time/60), :] = torch.FloatTensor(value_list)

        cell_index += 1
    
    globals()[f'x_tensor_{reg}'].size()
    
num_variables = globals()[f'x_tensor_{reg}'].shape[3]
time_length = globals()[f'x_tensor_{reg}'].shape[2]

41 43 40


torch.Size([124, 30, 120, 5])

torch.Size([124, 50, 120, 5])

torch.Size([124, 80, 120, 5])

In [5]:
y_df_add = find_directory(back_num = 2, foldername = ['Dataset', 'y_df.csv'])
y_df = pd.read_csv(y_df_add)

# 2. Regression problem
# KO, KP, CL
y_cl_numpy = y_df['Cycle life'].to_numpy()
y_kp_numpy = y_df['kneepoints'].to_numpy()
y_ko_numpy = y_df['kneeonsets'].to_numpy()
# y_ko_numpy.shape

y_regression_numpy = y_ko_numpy

y_ko_tensor = torch.FloatTensor(y_ko_numpy)
y_ko_tensor.size()

torch.Size([124])

In [6]:
def setRandomSeed(random_seed=0):
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    torch.manual_seed(random_seed) # torch 
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True # cudnn
    torch.backends.cudnn.benchmark = False # cudnn
    np.random.seed(random_seed) # numpy
    random.seed(random_seed) # random

### Dataloading as 2D CNN's method

In [7]:
# y: no scaling
class dataPrep_RNN_CNN(Dataset):  
    def __init__(self, x_tensor, y_tensor, batch_size, scaler, batches, test_split: float):
        setRandomSeed(random_seed = 100)
        self.xdata = torch.permute(x_tensor, (0, 3, 2, 1))
        self.ydata = y_tensor
        
        self.batch_size = batch_size
                
        self.xscaler = scaler()
        batches = [0] + batches
        self.batches = batches
        self.test_split = test_split
        
        # Tensor dimension: (cell, variable, time, cycle)
        # If problem emerges, check this part first.
        # Issue: 각 batch마다 따로 scaling을 해야하나?
        for i in range(len(self.xdata)):
            for j in range(self.xdata.shape[1]):
                temp = np.expand_dims(self.xscaler.fit_transform(self.xdata[i,j,:,:]), axis=0)
                if j ==0:
                    temp2 = temp
                else:
                    temp2 = np.vstack((temp2,temp))
            temp2 = np.expand_dims(temp2, axis=0)
            if i==0:
                self.xdata_scaled = temp2
            else:
                self.xdata_scaled = np.vstack((self.xdata_scaled, temp2))

        self.ydata_scaled = self.ydata
        
#         self.all = list(zip(self.xdata_scaled, self.ydata_scaled))
        
        prev = 0
        for i in range(1, len(self.batches)):
            globals()[f'self.b{i}_xdata_scaled'] = self.xdata_scaled[prev:prev + self.batches[i]]
            globals()[f'self.b{i}_ydata_scaled'] = self.ydata_scaled[prev:prev + self.batches[i]]
            
            prev +=self.batches[i]
            
            globals()[f'self.b{i}_train_size'], globals()[f'self.b{i}_val_size'], globals()[f'self.b{i}_test_size'] = self.get_splits_per_batch(self.batches[i])
            
            globals()[f'self.b{i}_x_train_scaled'], globals()[f'self.b{i}_x_test_scaled'], globals()[f'self.b{i}_y_train_scaled'], globals()[f'self.b{i}_y_test_scaled'] = train_test_split(globals()[f'self.b{i}_xdata_scaled'], 
                                                                                                            globals()[f'self.b{i}_ydata_scaled'], 
                                                                                                            test_size = globals()[f'self.b{i}_test_size'], 
                                                                                                            shuffle = True, 
                                                                                                            random_state=100)
        
            globals()[f'self.b{i}_x_train_scaled'], globals()[f'self.b{i}_x_val_scaled'], globals()[f'self.b{i}_y_train_scaled'], globals()[f'self.b{i}_y_val_scaled'] = train_test_split(globals()[f'self.b{i}_x_train_scaled'], 
                                                                                                            globals()[f'self.b{i}_y_train_scaled'], 
                                                                                                            test_size = globals()[f'self.b{i}_val_size'], 
                                                                                                            shuffle = True, 
                                                                                                            random_state=100)
            
            print(f'Batch {i}, total {self.batches[i]} cells')
            print('x_train, x_val, x_test:', globals()[f'self.b{i}_x_train_scaled'].shape, globals()[f'self.b{i}_x_val_scaled'].shape, globals()[f'self.b{i}_x_test_scaled'].shape)
            print('y_train, y_val, y_test:', globals()[f'self.b{i}_y_train_scaled'].shape, globals()[f'self.b{i}_y_val_scaled'].shape, globals()[f'self.b{i}_y_test_scaled'].shape)
        
            globals()[f'self.b{i}_train'] = list(zip(globals()[f'self.b{i}_x_train_scaled'], globals()[f'self.b{i}_y_train_scaled']))
            globals()[f'self.b{i}_val'] = list(zip(globals()[f'self.b{i}_x_val_scaled'], globals()[f'self.b{i}_y_val_scaled']))
            globals()[f'self.b{i}_test'] = list(zip(globals()[f'self.b{i}_x_test_scaled'], globals()[f'self.b{i}_y_test_scaled']))

            globals()[f'self.b{i}_train_dataloader'] = DataLoader(globals()[f'self.b{i}_train'], batch_size = self.batch_size, shuffle=True)
            globals()[f'self.b{i}_val_dataloader'] = DataLoader(globals()[f'self.b{i}_val'], batch_size = self.batch_size, shuffle=True)
            globals()[f'self.b{i}_test_dataloader'] = DataLoader(globals()[f'self.b{i}_test'], batch_size = self.batch_size, shuffle=True)
            
    # split the datasets into training and testing
    def get_splits_per_batch(self, batch_len):
        # test size
        test_size = int(self.test_split*batch_len)
        # val size
        val_size = int(0.2*(batch_len - test_size))
        # train_size
        train_size = batch_len - val_size - test_size
        return train_size, val_size, test_size
    
    # Returns dataloaders for training and validation datasets
    def train_data(self):
        self.train_dataloader = [globals()[f'self.b1_train_dataloader']]
        self.val_dataloader = [globals()[f'self.b1_val_dataloader']]
        if len(self.batches)>2:
            for i in range(2, len(self.batches)):
                self.train_dataloader.append(globals()[f'self.b{i}_train_dataloader'])
                self.val_dataloader.append(globals()[f'self.b{i}_val_dataloader'])
        return self.train_dataloader, self.val_dataloader
    
    # Returns dataloader for test datasets
    def test_data(self):
        self.test_dataloader = [globals()[f'self.b1_test_dataloader']]
        if len(self.batches)>2:
            for i in range(2, len(self.batches)):
                self.test_dataloader.append(globals()[f'self.b{i}_test_dataloader'])
        return self.test_dataloader
    
    def scaler(self):
        return self.xscaler
    
    # finding length of x 
    def __len__(self):
        return len(self.X)
    
    # indexing rows for calling
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]


In [8]:
# reg_dataset = dataPrep1D(x_tensor, y_ko_tensor, batch_size = 16, scaler = MinMaxScaler, purpose = 'regression', test_split = 0.2)
# reg_train_dataloader, reg_val_dataloader = reg_dataset.train_data()
# reg_test_dataloader = reg_dataset.test_data()
# reg_xscaler = reg_dataset.scaler('regression')

regression_cycles = [30, 50, 80]
# regression_cycles = [100]
batches = [b1, b2, b3]

# x와 y가 1대1 대응이어야 dataloader를 만들 수 있어서 2D array 그대로 놓고 학습시 tensor transform 예정
for reg in regression_cycles:
    print(reg, 'cycles')
    globals()[f'reg_dataset_{reg}'] = dataPrep_RNN_CNN(globals()[f'x_tensor_{reg}'], y_ko_tensor, batch_size = 4, scaler = MinMaxScaler, batches = batches, test_split = 0.2)
    globals()[f'reg_train_dataloader_{reg}'], globals()[f'reg_val_dataloader_{reg}'] = globals()[f'reg_dataset_{reg}'].train_data()
    globals()[f'reg_test_dataloader_{reg}'] = globals()[f'reg_dataset_{reg}'].test_data()
    globals()[f'reg_xscaler_{reg}'] = globals()[f'reg_dataset_{reg}'].scaler()
    for batch in range(len(batches)):
        globals()[f'b{batch+1}_train_dataloader_{reg}'], globals()[f'b{batch+1}_val_dataloader_{reg}'], globals()[f'b{batch+1}_test_dataloader_{reg}'] = globals()[f'reg_train_dataloader_{reg}'][batch], globals()[f'reg_val_dataloader_{reg}'][batch], globals()[f'reg_test_dataloader_{reg}'][batch]

30 cycles
Batch 1, total 41 cells
x_train, x_val, x_test: (27, 5, 120, 30) (6, 5, 120, 30) (8, 5, 120, 30)
y_train, y_val, y_test: torch.Size([27]) torch.Size([6]) torch.Size([8])
Batch 2, total 43 cells
x_train, x_val, x_test: (28, 5, 120, 30) (7, 5, 120, 30) (8, 5, 120, 30)
y_train, y_val, y_test: torch.Size([28]) torch.Size([7]) torch.Size([8])
Batch 3, total 40 cells
x_train, x_val, x_test: (26, 5, 120, 30) (6, 5, 120, 30) (8, 5, 120, 30)
y_train, y_val, y_test: torch.Size([26]) torch.Size([6]) torch.Size([8])
50 cycles
Batch 1, total 41 cells
x_train, x_val, x_test: (27, 5, 120, 50) (6, 5, 120, 50) (8, 5, 120, 50)
y_train, y_val, y_test: torch.Size([27]) torch.Size([6]) torch.Size([8])
Batch 2, total 43 cells
x_train, x_val, x_test: (28, 5, 120, 50) (7, 5, 120, 50) (8, 5, 120, 50)
y_train, y_val, y_test: torch.Size([28]) torch.Size([7]) torch.Size([8])
Batch 3, total 40 cells
x_train, x_val, x_test: (26, 5, 120, 50) (6, 5, 120, 50) (8, 5, 120, 50)
y_train, y_val, y_test: torch.Siz

## Define Model Class 

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [10]:
class RNNcell(nn.Module):
    def __init__(self, model, input_size, hidden_size, dropout = 0.1, bidirectional = False, num_layer=1):
        super(RNNcell, self).__init__()
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
            
        self.model = model
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.bidirectional = bidirectional
        
        self.D = 1 + self.bidirectional
        self.batch_size = 4
        
        #Batch size * cycles, timestep, input_size
        if self.model.endswith('RNN'):
            self.rnn = nn.RNN(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('LSTM'):
            self.rnn = nn.LSTM(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('GRU'):
            self.rnn = nn.GRU(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
            
        for name, param in self.rnn.named_parameters():
            if name.startswith('weight'):
                nn.init.xavier_uniform(param)
            else:
                nn.init.normal(param)
        
    def forward(self, x):
        # x made from (cell * n_cy, timestep(seq_len), num_vars)
        self.batch_size = x.size(0)
        
        D = 1 + self.bidirectional
        #out: Containing output features h_t from the last layer of LSTM for each t(if bidirectional ==True: concat of forward and backward hidden states)
        #h_n: Containing final hidden state in the sequence
        h0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
        if self.model.endswith('LSTM'):
            c0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
            # c_n: Containing final cell state in the sequence
            out, (hn, cn) = self.rnn(x, (h0, c0))
        else:
            out, hn = self.rnn(x, h0.detach())
        return out, hn

In [11]:
class SelfAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super(SelfAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_size = hidden_size // num_heads
        
        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, inputs):
        # inputs shape: [batch_size, seq_len, hidden_size]
        batch_size, seq_len, hidden_size = inputs.size()
        
        # Compute query, key, and value matrices
        query = self.query(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        key = self.key(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        value = self.value(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        
        # Compute scaled dot-product attention scores
        # attention_scores: [batch_size, num_heads, seq_len, seq_len]
        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.head_size, dtype=torch.float32))
        attention_probs = self.softmax(attention_scores)
        
        # Apply attention to the value matrix and concatenate heads
        # context: [batch_size, num_heads, seq_len, head_size]=>[batch_size, seq_len, num_heads, head_size] 
        # => [batch_size, seq_len, num_heads * head_size]
        context = torch.matmul(attention_probs, value).transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
#         context = nn.Linear(hidden_size, 1)(context)
        return attention_probs, context

In [12]:
class RNN_TA_CA_1DCNN(nn.Module):
    def __init__(self, input_size, num_time, num_cycles, rnn1, bi1, hid1, nhead2, fil2, pool2, npool2, fsize2, psize2, mids, dr1 = 0.1, dr2 = 0.1, di2 = 1, st2 = 1, pad2 = 0):
        super(RNN_TA_CA_1DCNN, self).__init__()
        setRandomSeed()
        
        self.D1 = 1 + bi1
        
        self.rnn1 = RNNcell(rnn1, input_size, hid1, dr1, bi1)
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        # Overall parameters
        self.num_time = num_time
        self.num_cycles = num_cycles
        self.mids = mids
        
        # Pooling/Nonpooling parameters
        # Pooling/Nonpooling layers
        self.pool2 = pool2
        self.npool2 = npool2
        # Pooling size(Pooling layer)
        self.psize2 = psize2
        
        # Params for self-attention(ca)
        self.nh2 = nhead2
        
        # CNN hyperparameters
        self.fil2 = fil2
        self.fsize2 = fsize2
        self.di2 = di2
        self.st2 = st2
        # Padding for CNN
        self.pad2 = pad2
                
        # Critical values for the 1st CNN layer
        # Batch for different cells
        
        # Conv1d takes input dimension: [Batch_size, c_in(no. of in_channels )= hidden_size, l_in = num_cycles]
        self.in2 = self.D1*hid1
        self.out2 = self.fil2
        self.Lout = self.num_cycles
        
        # Conv_block1~P
        # Pooling layers
        for i in range(1, self.pool2+1):
#             print(f"i: {i}, out_channels: {self.out_channels}")
            globals()[f'self.conv_block{i}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            nn.MaxPool1d(self.psize2)
            ).to(self.device)

            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int(int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)/self.psize2)

        for j in range(self.pool2+1, self.pool2+self.npool2+1):
            globals()[f'self.conv_block{j}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            ).to(self.device)
            
            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)
        
        self.lin1 = nn.Linear(self.D1*hid1, 1)
        self.sm1 = nn.Softmax(dim = 2)
        self.relu = nn.ReLU()
        
        self.ca = SelfAttention(self.D1*hid1, self.nh2)
        
        self.fc = nn.Sequential(nn.Linear(self.in2*self.Lout, self.mids[0], bias=True), nn.ReLU(),
                       nn.Linear(self.mids[0], self.mids[1], bias = True), nn.ReLU(),
                       nn.Linear(self.mids[1], self.mids[2], bias = True))
        
        self.fc.apply(self.init_weights)
        
    def init_weights(self, m):
        self.m = m
        if type(self.m) == nn.Linear:
            nn.init.xavier_uniform_(self.m.weight)
        
    def forward(self, x):
        # Original: Batch size, num_vars, timestep, cycles=>Batch size * cycles, timestep, input_size
#         print("Batch size, input_size, timestep, cycles: ", x.size())
        x = torch.reshape(x, (x.size(0)*x.size(3), x.size(2), x.size(1))).to(device)
#         print("Batch size * cycles, timestep, input_size: ", x.size())
        out1, hn1 = self.rnn1(x)
#         print('out1: (batch_size = num_cell*num_cycle, timestep, D1*hid1)', out1.size())
#         print(hn1.size())
        
        # hn의 batch size가 n_cy*k로 설정
        cell_batch_size = int(out1.size(0)/self.num_cycles)

        # Reshape for input
        # out1: (batch_size = num_cell*num_cycle, timestep, D1*hid1)
        # outs : (batch_size(num_cell), num_cycle, timestep, D1*hid1)
        outs1 = torch.reshape(out1, (cell_batch_size, -1, out1.size(1), out1.size(2)))
        # ta: (batch_size, num_cycle, timestep, 1(D1*hid1->1))
        ta = self.sm1(self.lin1(outs1))
        # ta_outs: (batch_size, num_cycle, timestep, D1*hid1)=>(batch_size, D1*hid1, num_cycle, timestep)
        ta_outs = torch.permute(ta*outs1, (0, 3, 1, 2))
        # ct_vec: (batch_size, D1*hid1, num_cycle)
        ct_vec = torch.sum(ta_outs, -1)
        # ct_vec shape: [batch_size, num_cycle, D1*hid1(hidden_size)]
        # hidden_size = num_heads * head_size
        # ct_vec transform back: [batch_size, num_heads, num_cycle, head_size]=>[batch_size, num_cycle, num_heads, head_size] 
        # => [batch_size, num_cycle, num_heads * head_size(hidden_size)]
        # ca: matmul(query[batch_size, num_heads, num_cycle, head_size], key^T[batch_size, num_heads, head_size, num_cycle])
        # ca shape: [batch_size, num_heads, num_cycle, num_cycle]
        ca, ct_vec = self.ca(ct_vec.transpose(2, 1))
#         print(f"After self attention: ca.size: {ca.size()}, ct_vec.size: {ct_vec.size()}")
        # ct_vec shape: [batch_size, D1*hid1, num_cycle]
        ct_vec = ct_vec.transpose(2, 1)
        
        
        # input for 1d cnn: (batch_size, D1*hid1, num_cycle)
        for i in range(1, self.pool2+self.npool2+1):
            ct_vec = globals()[f'self.conv_block{i}'](ct_vec)
        # ct_vec from final convolutional layer: (batch_size, self.in2*self.Lout)
        ct_vec = ct_vec.view(ct_vec.size(0), -1)
        # final_out: (batch_size, 1)
        final_out = self.fc(ct_vec)
        
        return final_out.squeeze(), ta.squeeze(), ca

In [13]:
# Function for saving and loading of training history
def save_data(D3_array, filename):
    with open(filename,"wb") as dat_:
        pickle.dump(D3_array,dat_)
        
def load_data(filename):
    with open(filename,"rb") as ld:
        x_temp = pickle.load(ld)
    return x_temp

# Early stopping class
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.best_epoch = 1
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, epoch, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_model = self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
#             print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_epoch = epoch
            self.best_model = self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'At epoch {self.best_epoch}: Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss
        return model

In [14]:
class train:
    def __init__(self, model: nn.Module, train_dataloader, val_dataloader, test_dataloader, epoch: int, learning_rate=0.01, patience = 5, verbose = False):
        super().__init__()
        #   Reprocudtion
        setRandomSeed(100)
        random_seed = 100
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(random_seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        os.environ['PYTHONHASHSEED'] = str(random_seed)
        torch.manual_seed(random_seed)
        np.random.seed(random_seed)

        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        self.model = model.to(self.device)
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.test_dataloader = test_dataloader
        self.epoch = epoch
        self.learning_rate = learning_rate
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = self.learning_rate)
        self.early_stopping = EarlyStopping(patience = patience, verbose = verbose)

        self.criterion = nn.MSELoss()
        self.name = ['train_rmse', 'val_rmse', 'test_rmse']
        
    def EvalModel(self):
        EvalPredictions, EvalActuals = list(), list()
        TestPredictions, TestActuals = list(), list()
        self.model.eval()
        with torch.no_grad():
            for n, (inputs, targets) in enumerate(self.val_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                yhat, ta, ca = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                if n==0:
                    EvalPredictions = yhat
                    EvalActuals = actual
                else:
                    EvalPredictions = np.concatenate((EvalPredictions, yhat), axis = 0)
                    EvalActuals = np.concatenate((EvalActuals, actual), axis = 0)

#             EvalPredictions, EvalActuals = vstack(EvalPredictions), vstack(EvalActuals)
            val_rmse = mean_squared_error(EvalActuals, EvalPredictions, squared = False)
            
            for n, (inputs, targets) in enumerate(self.test_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                yhat, ta, ca = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                    
                if n==0:
                    TestPredictions = yhat
                    TestActuals = actual
                else:
                    TestPredictions = np.concatenate((TestPredictions, yhat), axis = 0)
                    TestActuals = np.concatenate((TestActuals, actual), axis = 0)

#             TestPredictions, TestActuals = vstack(TestPredictions), vstack(TestActuals)
            test_rmse = mean_squared_error(TestActuals, TestPredictions, squared = False)
            
            return val_rmse, test_rmse

    def TrainModel(self):
        loss_history = []
        self.model.train()
        for i in range(self.epoch):
            for n, (inputs, targets) in enumerate(self.train_dataloader):
                self.model.train()
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                self.optimizer.zero_grad()

                yhat, ta, ca = self.model(inputs)
                    
                loss = self.criterion(yhat, targets)
                loss.backward()
                self.optimizer.step()
                
                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')
                    
                yhat = yhat.detach().numpy()
                targets = targets.numpy()
                
                if n==0:
                    predictions = yhat
                    actuals = targets
                else:
                    predictions = np.concatenate((predictions, yhat), axis = 0)
                    actuals = np.concatenate((actuals, targets), axis = 0)

            # evaluation of entire train data
#             predictions, actuals = np.append(predictions), np.append(actuals)
#             print("predictions: ", predictions)
#             print("actuals: ", actuals)
            
            train_rmse = mean_squared_error(actuals, predictions, squared = False)

            # evaluation on validation data
            val_rmse, test_rmse = self.EvalModel()
#             print("Batch ", i, train_rmse, val_rmse, test_rmse)
            early_factor = val_rmse
            
            # training loss history
            loss_history.append([train_rmse, val_rmse, test_rmse])

            self.early_stopping(i+1, early_factor, self.model)
            
            if self.early_stopping.early_stop:
                print("Early stopping at best epoch: ", self.early_stopping.best_epoch)
                self.best_loss = loss_history[self.early_stopping.best_epoch-1]
                loss_history = loss_history[:self.early_stopping.best_epoch]
                
                return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta, ca
            
        print(f"Ends at final epoch {self.epoch}")
        print(f"Best epoch: {self.early_stopping.best_epoch}")
        self.best_loss = loss_history[self.early_stopping.best_epoch-1]
        loss_history = loss_history[:self.early_stopping.best_epoch]
        
        return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta, ca
        
    def predict(self, data: torch.Tensor):
        return self.model(data)
    
    def train_val_test_plot_rmse(self, num_cycles, modelname, n_ep, patience, rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr):
        dataloaders = [self.train_dataloader, self.val_dataloader, self.test_dataloader]

        with torch.no_grad():
            model.eval()

            model_rmse = list()
            dset_index = 0
            for dataloader in dataloaders:
                if dset_index ==0:
                    dset = 'train'
                elif dset_index == 1:
                    dset = 'val'
                else:
                    dset = 'test'

                figadd = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.jpg')

                predictions, actuals = list(), list()
                for i, (test_input, test_target) in enumerate(dataloader):
                    test_input = test_input.float().to(self.device)
                    test_target = test_target.float().to(self.device)
                    test_yhat, test_ta = model(test_input)

                    if self.device != 'cpu':
                        test_yhat = test_yhat.to('cpu')
                        test_target = test_target.to('cpu')

                    test_yhat = test_yhat.detach().tolist()
                    predictions += test_yhat
                    test_target = test_target.tolist()
                    actuals += test_target

                pred_acc, targ_acc = np.array(predictions), np.array(actuals)

                rmse = mean_squared_error(actuals, predictions, squared = False)
                model_rmse.append(rmse)

                plt.figure(figsize=[10,4])
                plt.rcParams["font.size"] = "12"
                plt.plot(pred_acc, label='Prediction')
                plt.plot(range(len(targ_acc)), targ_acc, label = 'Actual')
                plt.xticks(range(0, len(targ_acc) + 1, int(len(targ_acc)/5)))
                plt.title(f"{rnn1}_hid_{hid1}_nfil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}: RMSE={rmse}")
                plt.xlabel('cell')
                plt.ylabel('Lifespan(cycle)')
                plt.legend()
                plt.savefig(figadd)
                plt.close()

                dset_index += 1

        return model_rmse

## Implementation

In [15]:
def history_state_dict_add(num_cycles, modelname, n_ep, patience, rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr, batch_num):
    history_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', f'batch_{batch_num}', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.pkl')
    state_dict_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', f'batch_{batch_num}', 'model'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_state_dict.pth')
    ta_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', f'batch_{batch_num}', 'ta'],
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    ca_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', f'batch_{batch_num}', 'ca'],
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    return history_add, state_dict_add, ta_add, ca_add

## Each batch

In [ ]:
# Hyperparameters
num_vars = globals()[f'x_tensor_{reg}'].size()[3]

modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycless = [30, 50, 80]
ep_pats = [[3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [2, 3, 4, 5]

batch_nums = [1]

import time

for batch_num in batch_nums:
    for [n_ep, patience] in ep_pats:
        for num_cycles in num_cycless:
            print(f"Batch: {batch_num + 1}, num_cycle = {num_cycles}, epoch = {n_ep}, patience = {patience}")
            trdl = globals()[f'b{batch_num+1}_train_dataloader_{num_cycles}']
            vdl = globals()[f'b{batch_num+1}_val_dataloader_{num_cycles}']
            tedl = globals()[f'b{batch_num+1}_test_dataloader_{num_cycles}']
            for rnn1, hid1 in itertools.product(rnns, hids):
                bi1 = False
                if rnn1.startswith('Bi'):
                    bi1 = True
                for pool2, npool2 in itertools.product(pools, npools):
                    print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers, split by {60/max_time} min')
                    for fil2, lr in itertools.product(num_fils, lrs):
                        for nh2 in nh2s:
                            print(f"rnn = {rnn1}, bi = {bi1}, hid per head = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")

                            all_hid1 = hid1 * nh2

                            start = time.time()

                            # Construct CNN
                            model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids).to(device)

                            # Train
                            model_train = train(model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False)
                            best_model, best_loss, history, ta, ca = model_train.TrainModel()

                            print('best_loss = '+ str(best_loss))

                            history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                        rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr, batch_num+1)

                            # saving best_epoch, loss history, ta score
                            save_data(history, history_add)
                            save_data(best_model.state_dict(), state_dict_add)
                            save_data(ta, ta_add)
                            save_data(ca, ca_add)

                            print("time: ", time.time()-start)
                            print("\n\n")

Batch: 2, num_cycle = 30, epoch = 3000, patience = 500
Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 3, lr = 0.0001
Ends at final epoch 3000
Best epoch: 2976
best_loss = [33.250347, 21.251219, 73.60257]
time:  120.46832275390625



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1495
best_loss = [28.095877, 14.428834, 85.06497]
time:  80.05868721008301



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  928
best_loss = [36.613544, 17.491817, 133.4673]
time:  58.07500696182251



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 3, lr = 0.0001
Ends at final epoch 3000
Best epoch: 3000
best_loss = [292.78732, 293.05704, 314.22647]
time:  125.02721786499023



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 3, lr = 0.001
Early stopping at best epoch

Early stopping at best epoch:  89
best_loss = [39.888763, 12.996736, 150.3829]
time:  24.360878705978394



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  407
best_loss = [35.38578, 11.897153, 105.35777]
time:  37.60534334182739



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 3, lr = 0.01
Early stopping at best epoch:  113
best_loss = [47.120583, 13.219053, 103.37961]
time:  24.70645260810852



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  147
best_loss = [60.876064, 12.45855, 62.355156]
time:  27.44455599784851



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 3, lr = 0.01
Early stopping at best epoch:  368
best_loss = [27.228388, 12.801936, 129.34767]
time:  38.6567223072052



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  79
best_loss = [22.749489, 

Early stopping at best epoch:  1414
best_loss = [54.75804, 13.2489805, 96.9507]
time:  83.49617743492126



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  938
best_loss = [29.38669, 20.182493, 95.838036]
time:  61.67370104789734



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1895
best_loss = [36.661335, 11.379465, 136.23]
time:  104.43706798553467



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 5, lr = 0.001
Early stopping at best epoch:  532
best_loss = [46.50645, 6.8059626, 86.9898]
time:  44.0484676361084



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 5, lr = 0.001
Early stopping at best epoch:  151
best_loss = [43.605545, 14.152275, 108.55205]
time:  28.248472452163696



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 5, lr = 0.001
Early stopping at best epoch:  368
best_loss = [23.147

Early stopping at best epoch:  173
best_loss = [29.287472, 13.143114, 144.25116]
time:  30.20868754386902



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 5, lr = 0.01
Early stopping at best epoch:  86
best_loss = [63.324696, 7.6868377, 84.1173]
time:  25.850192546844482



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 5, lr = 0.01
Early stopping at best epoch:  141
best_loss = [49.66373, 10.233472, 97.11534]
time:  28.37588119506836



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  67
best_loss = [66.07692, 10.113034, 96.42492]
time:  25.266980409622192



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  1205
best_loss = [58.29167, 17.592102, 94.329704]
time:  79.0470654964447



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  1078
best_loss = [83.430084

Early stopping at best epoch:  786
best_loss = [31.133013, 15.413713, 147.06491]
time:  52.39836835861206



rnn = LSTM, bi = False, hid per head = 5, num_heads = 2, num_fil = 7, lr = 0.001
Early stopping at best epoch:  189
best_loss = [53.818718, 11.784113, 86.522705]
time:  27.50518274307251



rnn = LSTM, bi = False, hid per head = 5, num_heads = 3, num_fil = 7, lr = 0.001
Early stopping at best epoch:  375
best_loss = [46.640438, 10.293264, 108.81723]
time:  35.22942066192627



rnn = LSTM, bi = False, hid per head = 5, num_heads = 4, num_fil = 7, lr = 0.001
Early stopping at best epoch:  189
best_loss = [42.67016, 10.070509, 87.797356]
time:  28.061315298080444



rnn = LSTM, bi = False, hid per head = 5, num_heads = 5, num_fil = 7, lr = 0.001
Early stopping at best epoch:  516
best_loss = [51.460167, 12.029634, 86.05437]
time:  40.897485971450806



rnn = LSTM, bi = False, hid per head = 5, num_heads = 2, num_fil = 7, lr = 0.01
Early stopping at best epoch:  24
best_loss = [55.6

...

Early stopping at best epoch:  1156
best_loss = [60.550095, 24.097006, 102.58316]
time:  84.70649027824402



rnn = GRU, bi = False, hid per head = 5, num_heads = 3, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  1487
best_loss = [46.374172, 23.318638, 104.66483]
time:  102.21699547767639



rnn = GRU, bi = False, hid per head = 5, num_heads = 4, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  357
best_loss = [74.09387, 21.188864, 92.2999]
time:  44.19501090049744



rnn = GRU, bi = False, hid per head = 5, num_heads = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  687
best_loss = [42.74365, 23.605864, 94.48608]
time:  61.97632169723511



rnn = GRU, bi = False, hid per head = 5, num_heads = 2, num_fil = 7, lr = 0.001
Early stopping at best epoch:  175
best_loss = [49.68463, 16.543575, 89.85571]
time:  34.1888964176178



rnn = GRU, bi = False, hid per head = 5, num_heads = 3, num_fil = 7, lr = 0.001
Early stopping at best epoch:  549
best_loss = [38.913334

Early stopping at best epoch:  30
best_loss = [49.158855, 16.891804, 69.00359]
time:  25.6345374584198



rnn = GRU, bi = False, hid per head = 7, num_heads = 3, num_fil = 7, lr = 0.01
Early stopping at best epoch:  143
best_loss = [11.504722, 19.375967, 113.73265]
time:  31.464325189590454



rnn = GRU, bi = False, hid per head = 7, num_heads = 4, num_fil = 7, lr = 0.01
Early stopping at best epoch:  21
best_loss = [54.800312, 15.729363, 83.13119]
time:  25.60503888130188



rnn = GRU, bi = False, hid per head = 7, num_heads = 5, num_fil = 7, lr = 0.01
Early stopping at best epoch:  13
best_loss = [57.687286, 24.471563, 103.046394]
time:  27.20557737350464



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = GRU, bi = False, hid per head = 7, num_heads = 2, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  700
best_loss = [47.039913, 22.07436, 106.33447]
time:  60.907230615615845



rnn = GRU, bi = False, hid per head = 7, num_heads = 3, num_fil = 3, lr = 0.0001
Ea

Early stopping at best epoch:  609
best_loss = [35.72156, 53.91613, 148.62009]
time:  61.0880012512207



rnn = GRU, bi = False, hid per head = 7, num_heads = 2, num_fil = 3, lr = 0.001
Early stopping at best epoch:  106
best_loss = [49.011395, 24.88248, 111.54147]
time:  31.224948406219482



rnn = GRU, bi = False, hid per head = 7, num_heads = 3, num_fil = 3, lr = 0.001
Early stopping at best epoch:  465
best_loss = [60.74556, 13.0969, 108.069595]
time:  50.315157890319824



rnn = GRU, bi = False, hid per head = 7, num_heads = 4, num_fil = 3, lr = 0.001
Early stopping at best epoch:  66
best_loss = [53.978966, 35.49444, 98.56274]
time:  29.806878566741943



rnn = GRU, bi = False, hid per head = 7, num_heads = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  142
best_loss = [55.080555, 15.659807, 72.04008]
time:  35.7099928855896



rnn = GRU, bi = False, hid per head = 7, num_heads = 2, num_fil = 3, lr = 0.01
Early stopping at best epoch:  18
best_loss = [56.661186, 24.527

Early stopping at best epoch:  450
best_loss = [53.24391, 5.068868, 98.429504]
time:  53.91724634170532



rnn = GRU, bi = False, hid per head = 7, num_heads = 2, num_fil = 5, lr = 0.0001
Ends at final epoch 3000
Best epoch: 3000
best_loss = [293.1574, 293.43405, 314.58868]
time:  166.8294129371643



rnn = GRU, bi = False, hid per head = 7, num_heads = 3, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  908
best_loss = [51.47674, 20.754068, 90.19801]
time:  78.02814412117004



rnn = GRU, bi = False, hid per head = 7, num_heads = 4, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  458
best_loss = [61.621666, 24.3302, 92.803635]
time:  53.49065899848938



rnn = GRU, bi = False, hid per head = 7, num_heads = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  587
best_loss = [43.820778, 18.042385, 79.69157]
time:  63.66238498687744



rnn = GRU, bi = False, hid per head = 7, num_heads = 2, num_fil = 5, lr = 0.001
Ends at final epoch 3000
Best epoch: 3000
best_loss 

Early stopping at best epoch:  424
best_loss = [38.628456, 16.265593, 102.82079]
time:  46.85869216918945



rnn = RNN, bi = False, hid per head = 3, num_heads = 2, num_fil = 5, lr = 0.01
Early stopping at best epoch:  507
best_loss = [59.98549, 17.735626, 104.729675]
time:  50.30143404006958



rnn = RNN, bi = False, hid per head = 3, num_heads = 3, num_fil = 5, lr = 0.01
Early stopping at best epoch:  66
best_loss = [37.914963, 17.87546, 100.93974]
time:  28.1198091506958



rnn = RNN, bi = False, hid per head = 3, num_heads = 4, num_fil = 5, lr = 0.01
Early stopping at best epoch:  150
best_loss = [41.4485, 17.059765, 91.59106]
time:  32.65036582946777



rnn = RNN, bi = False, hid per head = 3, num_heads = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  11
best_loss = [62.633404, 20.592411, 94.629036]
time:  25.905970096588135



rnn = RNN, bi = False, hid per head = 3, num_heads = 2, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  421
best_loss = [60.84629, 24.455

In [ ]:
# Hyperparameters
num_vars = globals()[f'x_tensor_{reg}'].size()[3]

modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycless = [30, 50, 80]
ep_pats = [[1000, 10], [2000, 20]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [1, 2, 3, 4, 5]

batch_nums = [1]

import time

for batch_num in batch_nums:
    for [n_ep, patience] in ep_pats:
        for num_cycles in num_cycless:
            print(f"Batch: {batch_num + 1}, num_cycle = {num_cycles}, epoch = {n_ep}, patience = {patience}")
            trdl = globals()[f'b{batch_num+1}_train_dataloader_{num_cycles}']
            vdl = globals()[f'b{batch_num+1}_val_dataloader_{num_cycles}']
            tedl = globals()[f'b{batch_num+1}_test_dataloader_{num_cycles}']
            for rnn1, hid1 in itertools.product(rnns, hids):
                bi1 = False
                if rnn1.startswith('Bi'):
                    bi1 = True
                for pool2, npool2 in itertools.product(pools, npools):
                    print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers, split by {60/max_time} min')
                    for fil2, lr in itertools.product(num_fils, lrs):
                        for nh2 in nh2s:
                            print(f"rnn = {rnn1}, bi = {bi1}, hid per head = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")

                            all_hid1 = hid1 * nh2

                            start = time.time()

                            # Construct CNN
                            model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids).to(device)

                            # Train
                            model_train = train(model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False)
                            best_model, best_loss, history, ta, ca = model_train.TrainModel()

                            print('best_loss = '+ str(best_loss))

                            history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                        rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr, batch_num+1)

                            # saving best_epoch, loss history, ta score
                            save_data(history, history_add)
                            save_data(best_model.state_dict(), state_dict_add)
                            save_data(ta, ta_add)
                            save_data(ca, ca_add)

                            print("time: ", time.time()-start)
                            print("\n\n")

In [ ]:
modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [1, 2, 3, 4, 5]

batch_nums = [0, 2, 1]

for batch_num in batch_nums:
    for [n_ep, patience] in ep_pats:
        for nh2 in nh2s:
            for rnn1 in rnns:
                df = pd.DataFrame(columns = ['train rmse', 'val rmse', 'test rmse'])
                bi1 = False
                for hid1 in hids:
                    for pool2, npool2 in itertools.product(pools, npools):
                        for fil2, lr in itertools.product(num_fils, lrs):
                            history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                        rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr, batch_num+1)

                            history = load_data(history_add)

                            df.loc[f'Batch {batch_num+1}_{nh2}_{rnn1}_{hid1}_{fil2}_{pool2}_{npool2}_{fsize2}_{lr}'] = [history.iloc[-1, 0], history.iloc[-1, 1], history.iloc[-1, 2]]

                df
                df_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, 'Each batch', f'Batch {batch_num+1}'], 
                                filename = f'Batch {batch_num+1}_{nh2} heads_{modelname}_{rnn1}_{nh2}.csv')
                df.to_csv(df_add)